In [1]:
#setup credentials for BigQuery
from google.cloud import bigquery
credentials_path = './test-bigquery-ot-dcacff312913.json'
client = bigquery.Client.from_service_account_json(credentials_path)

In [2]:
#Imports
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
# Pull in the 1000 genomes pedigree information from a public BigQuery table

query='''
SELECT
    study_id,
    trait_reported
FROM
    `bigquery-public-data.open_targets_genetics.studies`
WHERE
    LOWER(trait_reported) LIKE '%coeliac%' AND
    study_id LIKE '%NEALE2%' AND
    study_id NOT LIKE '%raw%'
LIMIT 10
'''

# Run the query
query_job = client.query(query)

# Get the results as a pandas dataframe
study = query_job.to_dataframe()
study

,study_id,trait_reported
0,NEALE2_20002_1456,Malabsorption/coeliac disease | non-cancer ill...


In [4]:
query='''
SELECT
    *
FROM
    `open-targets-genetics.genetics.locus2gene`
WHERE
    study_id = 'NEALE2_20002_1456' AND
    y_proba_full_model > 0.2
LIMIT 10
'''

# Run the query
query_job = client.query(query)

l2g = query_job.to_dataframe()
l2g

,study_id,chrom,pos,ref,alt,gene_id,training_clf,training_gs,training_fold,y_proba_dist_foot,y_proba_dist_tss,y_proba_full_model,y_proba_logi_distance,y_proba_logi_interaction,y_proba_logi_molecularQTL,y_proba_logi_pathogenicity,y_proba_logo_distance,y_proba_logo_interaction,y_proba_logo_molecularQTL,y_proba_logo_pathogenicity
0,NEALE2_20002_1456,2,181233267,T,A,ENSG00000170035,xgboost,high_medium,fold3=12|2|7|17,0.618257,0.152766,0.549886,0.688754,0.613868,0.218280,0.222992,0.127543,0.508816,0.674206,0.645418
1,NEALE2_20002_1456,3,159929885,A,G,ENSG00000151967,xgboost,high_medium,fold0=8|1|3,0.503329,0.244470,0.305152,0.393564,0.046665,0.038898,0.072347,0.071021,0.377236,0.326806,0.381904
2,NEALE2_20002_1456,3,188362632,C,G,ENSG00000145012,xgboost,high_medium,fold0=8|1|3,0.503358,0.560062,0.608407,0.648355,0.476584,0.404791,0.513987,0.546520,0.679430,0.610003,0.621999
3,NEALE2_20002_1456,3,46150778,T,C,ENSG00000163823,xgboost,high_medium,fold0=8|1|3,0.469798,0.234810,0.359715,0.339675,0.389763,0.152666,0.311378,0.315121,0.369243,0.338774,0.340413
4,NEALE2_20002_1456,3,69206160,T,C,ENSG00000163380,xgboost,high_medium,fold0=8|1|3,0.469798,0.560062,0.221527,0.207005,0.030283,0.033275,0.058126,0.057849,0.266176,0.225643,0.190149
5,NEALE2_20002_1456,6,33568227,G,T,ENSG00000030110,xgboost,high_medium,fold4=13|6|19|15|14,0.538778,0.577258,0.394328,0.343315,0.094812,0.035294,0.501866,0.437024,0.353659,0.409501,0.272308
6,NEALE2_20002_1456,3,69206160,T,C,ENSG00000114541,xgboost,high_medium,fold0=8|1|3,0.503358,0.233411,0.317729,0.344943,0.212655,0.344844,0.688858,0.647631,0.292910,0.376869,0.292376
7,NEALE2_20002_1456,6,137652695,C,T,ENSG00000118503,xgboost,high_medium,fold4=13|6|19|15|14,0.319008,0.126758,0.265522,0.385325,0.420626,0.161637,0.157325,0.221851,0.261481,0.347296,0.267648
8,NEALE2_20002_1456,4,122618699,G,A,ENSG00000109471,xgboost,high_medium,fold2=11|18|20|4|9,0.026932,0.014773,0.256677,0.345844,0.230633,0.154625,0.220051,0.210597,0.235742,0.383207,0.306041
9,NEALE2_20002_1456,18,12805389,G,A,ENSG00000175354,xgboost,high_medium,fold2=11|18|20|4|9,0.576733,0.283069,0.671598,0.770825,0.177920,0.687255,0.693599,0.694059,0.643880,0.797773,0.781645


In [5]:
#I appear to have hit some sort of quota with the disease_variant_gene table

query='''
SELECT
  study_id
FROM
  `open-targets-genetics.genetics.disease_variant_gene`
WHERE
    study_id = 'NEALE2_20002_1456'
LIMIT
  5
'''

# Run the query
query_job = client.query(query)

lead_snp_info = query_job.to_dataframe()
lead_snp_info

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: EU
Job ID: 7ff4e0c5-a816-4141-83a0-655db8891a8e


In [6]:
query='''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model,
        lead_snp_info.pval, lead_snp_info.beta, lead_snp_info.odds_ratio
    
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    
    #get GWAS metadata
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    
    #get HGNC IDs
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id
    
    #get lead SNP P-values
    INNER JOIN `bigquery-public-data.open_targets_genetics.disease_variant_gene` AS lead_snp_info
    ON locus2gene.study_id = lead_snp_info.study_id 
        AND locus2gene.chrom = lead_snp_info.lead_chrom
        AND locus2gene.pos = lead_snp_info.lead_pos
        AND locus2gene.ref = lead_snp_info.lead_ref
        AND locus2gene.alt = lead_snp_info.lead_alt
        #AND locus2gene.gene_id = lead_snp_info.gene_id
    
    #filter on the Neale Lab UKBB analysis
    WHERE locus2gene.study_id LIKE '%NEALE2%'
        
        #optional: filter on a disease
        AND LOWER(study_metadata.trait_reported) LIKE '%coeliac%'
        
        #remove the "raw" neale lab results -- I'm not sure what this is
        AND locus2gene.study_id NOT LIKE '%raw%'

        #filter to a l2g score threshold
        AND locus2gene.y_proba_full_model > 0.2


    )
SELECT * from ranked_genes
LIMIT 10
'''

# Run the query
query_job = client.query(query)

l2g = query_job.to_dataframe()
l2g

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 3fc5d200-626b-41ae-8f51-1e01de4128fe


In [8]:
query='''
WITH ranked_genes AS (
    SELECT locus2gene.study_id, 
        locus2gene.chrom, locus2gene.pos, locus2gene.ref, locus2gene.alt, 
        study_metadata.*, 
        genes.gene_name, locus2gene.y_proba_full_model
    
    FROM `bigquery-public-data.open_targets_genetics.locus2gene` AS locus2gene
    
    #get GWAS metadata
    INNER JOIN `bigquery-public-data.open_targets_genetics.studies` AS study_metadata
    ON locus2gene.study_id = study_metadata.study_id
    
    #get HGNC IDs
    INNER JOIN `bigquery-public-data.open_targets_genetics.genes` AS genes
    ON locus2gene.gene_id = genes.gene_id

    
    #filter on the Neale Lab UKBB analysis
    WHERE locus2gene.study_id LIKE '%NEALE2%'
        
        #optional: filter on a disease
        AND LOWER(study_metadata.trait_reported) LIKE '%coeliac%'
        
        #remove the "raw" neale lab results -- I'm not sure what this is
        AND locus2gene.study_id NOT LIKE '%raw%'

        #filter to a l2g score threshold
        AND locus2gene.y_proba_full_model > 0.2

    )
SELECT * from ranked_genes
'''

# Run the query
query_job = client.query(query)

l2g = query_job.to_dataframe()
l2g

,study_id,chrom,pos,ref,alt,study_id_1,ancestry_initial,ancestry_replication,n_cases,n_initial,...,pub_journal,pub_title,has_sumstats,num_assoc_loci,source,trait_reported,trait_efos,trait_category,gene_name,y_proba_full_model
0,NEALE2_20002_1456,3,46150778,T,C,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,CCR1,0.359715
1,NEALE2_20002_1456,3,69206160,T,C,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,LMOD3,0.221527
2,NEALE2_20002_1456,6,33746343,G,A,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,IP6K3,0.629454
3,NEALE2_20002_1456,3,159929885,A,G,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,IL12A,0.496580
4,NEALE2_20002_1456,4,122618699,G,A,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,IL21,0.416285
5,NEALE2_20002_1456,2,181233267,T,A,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ITGA4,0.236174
6,NEALE2_20002_1456,6,28469986,AGAG,A,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,ZSCAN12,0.376536
7,NEALE2_20002_1456,3,188362632,C,G,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,LPP,0.608407
8,NEALE2_20002_1456,6,33568227,G,T,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,BAK1,0.394328
9,NEALE2_20002_1456,3,69206160,T,C,NEALE2_20002_1456,{'list': [{'element': 'European=361141'}]},{'list': []},1587,361141,...,None,None,True,21,NEALE,Malabsorption/coeliac disease | non-cancer ill...,"{'list': [{'element': 'HP_0002024'}, {'element...",gastrointestinal disease,FRMD4B,0.317729
